## Exercise: The inverted index

The inverted index is the data structure that drives unstructured ad-hoc text retrieval, and it has no real competitors. The inverted index makes searching at web scale (billions of documents) possible and efficient. 

In this exercise you are going to implement an inverted index and use it to retrieve songs by lyrics from the *One Million Song* data set. Because we use a high-level interpreted language and do not pay painstaking attention to code efficiency, the code you develop in this exercise may not be completely ready to take on the whole web. But you will know the basic principles.

### Data set 

Download [the lyrics in TSV format](https://dl.dropboxusercontent.com/u/1423772/sciprog/mxm_dataset_preprocessed.tsv)  and place the file in the current directory. The data has a line for each of about 779,000 songs. The first column is a song id, followed by artist name and song title. The last column of the line contains the lyrics of the song. It's in a bag-of-words format, meaning that the original linear sequence of the words is not preserved. Each word is associated with a count - the number of times it occured in the original lyrics. Below is an example of a line from the file:

```
1       Western Addiction       A Poor Recipe for Civic Cohesion        i:6 the:4 you:2 to:2 and:5 a:3 me:1 it:1 my:1 is:2 of:3 your:1 that:1 are:2 we:2 am:2 will:2 for:4 be:1 have:2 so:1 this:1 like:2 de:1 up:1 was:2 if:1 got:1 would:1 been:1 these:2 seems:1 someone:1 understand:1 pass:1 river:1 met:1 piece:1 damn:1 worth:1 flesh:1 grace:1 poor:2 somehow:1 ignore:1 passion:1 tide:1 season:1 seed:1 resist:1 order:2 pity:1 fashion:1 granted:1 captured:2 ici:1 soil:1 patience:1 social:2 highest:2 slice:1 leaf:1 lifeless:1 arranged:1 wilderness:1 shark:1 devastation:1 elements:1
```


### Term index

Your task is now to create an inverted index that permits efficient keyword searching of the song collection. An inverted index in this context maps from the words of the songs to the song ids. So for every word that is used in the songs you should be able to lookup quickly which songs it is being used in.

This data structure is also refered to as a **term index**, where words of the songs are considered terms. The lists of song ids are called **posting lists**. 

In [2]:
import codecs
from itertools import islice
from collections import defaultdict

terms = defaultdict(list)
doc_metadata = defaultdict(dict)

words = []
for line in islice(open("mxm_dataset_preprocessed.tsv"), 1000):
    parts = line.split("\t")

    # YOUR CODE HERE

# Sort posting lists
for postings in terms.values():
    postings.sort()

### Ex. 2.1. Conjunctive query 

Write a function `query_conj(query)` that accepts a string parameter `query` of white-space separated terms. `query_conj` should intersect the postings lists associated with the query terms and return the results. You should not use the Python `set` operations for this purpose, because this approach will not scale for large data sets. 

When you have three or more query terms, the intersection operation can be decomposed into binary operations involving only two postings lists. For instance, with postings lists $p, q, r$, you could intersect $p$ and $q$ in a temporary list $p'$, which could then be intersected with $r$. So the binary operations in this example would be $p' = p \cap q$ and $p' \cap r$.

Test the speed of your implementation with the `%%timeit` IPython magic. Place the magic in the top of a cell and write the code you want to have benchmarked below. For instance, 

```
%%timeit
query_conj("love and gray silver")

```

Try queries that involve both large and small intersections. 

In [40]:
# Your code here

### Ex 2.2 Conjunctive query with binary search

In the exercise above you might have noticed that a lot of unnecessary work is done when intersecting a small postings list like that for *pan* and huge ones for common words like *and*. In a nutshell the problem is that the large list has an entry for almost every document, while there may be thousands of documents between entries in the smaller list. And yet we dutifully check every entry in the larger list against the smaller list. Is it possible to skip ahead?

Yes, because both lists are sorted. We can thus jump some distance ahead in the larger list, checking 

1. whether we overshoot, 
2. are still not far enough ahead, or 
3. perhaps got lucky and landed exactly at the correct element in the larger list. 

In case of 1) or 2), we make another jump to get closer to the correct element. In case the jump overshot, we know that the value we are searching for must be between the initial value and the place we landed. This gives us a search space. If every jump halves the search space, this approach is called binary search. It may happen that the larger list does not contain the current entry of the smaller list. We can detect this situation when the search space spans nothing. 

Binary search may be one of the simplest algorithm out there, but despite its conceptual simplicity it needs careful implementation.

Implement a conjunctive query with binary search `query_conj_bs` and test the performance against `query_conj` on several different queries. 

In [41]:
# Your code here